In [56]:
import pandas as pd
import os
%config Completer.use_jedi = False
csvpath = os.path.join("HeroesOfPymoli","Resources","purchase_data.csv")

purchase_data = pd.read_csv(csvpath)

pymoli_df = pd.DataFrame(purchase_data)

pymoli_df.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [57]:
#count of players
player_count_df = pymoli_df["SN"].nunique()
playerdict = [{"Total Players" : (player_count_df)}]
totalplayers = pd.DataFrame(playerdict)
totalplayers

,Total Players
0,576


In [58]:
#Count of unique items
item_types_df = pymoli_df["Item ID"].nunique()

#Calculate average purchase price
ave_price = pymoli_df["Price"].mean()
ave_price_currency = "${:.2f}".format(ave_price)
#Total number of purchases
num_purchases = pymoli_df["Purchase ID"].count()

#calculate total Revenue
total_revenue = pymoli_df["Price"].sum()
total_revenue_currency = "${:.2f}".format(total_revenue)
#create dictionary
purchasing_analysis_dict = [{"Number of Unique Items" : (item_types_df), "Average Price" : (ave_price_currency), "Number of Purchases": (num_purchases), "Total Revenue" : (total_revenue_currency)}]

purchasing_analysis_df = pd.DataFrame(purchasing_analysis_dict)

purchasing_analysis_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [59]:
#gender demos and total percentage
player_count = len(purchase_data["SN"].unique())

#players
male_players = purchase_data.loc[purchase_data["Gender"] == "Male"]
male_count = len(male_players["SN"].unique())
male_percent = "{:.2f}%".format(male_count / player_count * 100)

female_players = purchase_data.loc[purchase_data["Gender"] == "Female"]
female_count = len(female_players["SN"].unique())
female_percent = "{:.2f}%".format(female_count / player_count * 100)

other_players = purchase_data.loc[purchase_data["Gender"] == "Other / Non-Disclosed"]
other_count = len(other_players["SN"].unique())
other_percent = "{:.2f}%".format(other_count / player_count * 100)

#summary
gender_demographics_table = pd.DataFrame([{
    "Gender": "Male", "Total Count": male_count, 
    "Percentage of Players": male_percent}, 
    {"Gender": "Female", "Total Count": female_count, 
     "Percentage of Players": female_percent}, 
    {"Gender": "Other / Non-Disclosed", "Total Count": other_count, 
     "Percentage of Players": other_percent
    }], columns=["Gender", "Total Count", "Percentage of Players"])

gender_demographics_table = gender_demographics_table.set_index("Gender")
gender_demographics_table.index.name = "Gender"
gender_demographics_table



,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [60]:
#create grouped data frame
gender_grouped_df = pymoli_df.copy()
gender_grouped_df = pymoli_df.groupby(["Gender"])
gpurchase_count = gender_grouped_df["Purchase ID"].count()
gprice_ave = gender_grouped_df["Price"].mean().map("${:.2f}".format)
gtotal_value =gender_grouped_df["Price"].sum().map("${:.2f}".format)

gendertotals = gender_grouped_df["Price"].sum()
individuals = gender_grouped_df["SN"].nunique()
ave_per_player = (gendertotals/individuals).map("${:.2f}".format)
#combine columns for display
gender_analysis_df = pd.DataFrame({"Purchase Count" : gpurchase_count, "Average Purchase Price" : gprice_ave, "Total Purchase Value" : gtotal_value, "Avg Total Purchase Per Person" : ave_per_player})

gender_analysis_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [61]:
#create a copy data frame to work with
player_age_df = pymoli_df.copy()
#create bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
binlabels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_groups = pd.cut(player_age_df["Age"], bins, labels = binlabels)
player_age_df["Age Bracket"] = age_groups
player_age_df.drop_duplicates(["SN"], inplace=True)

player_ages = player_age_df["Age Bracket"].value_counts()
#add percentage of players
percents = player_age_df["Age Bracket"].value_counts(normalize=True)*100
percent_age = (percents).map("{:.2f}%".format)
#combine and sort age and percentage for display
age_df = pd.DataFrame({"Total Count" : player_ages, "Percentage of Players" : percent_age})
age_analysis = age_df.sort_index()
age_analysis

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [62]:
#create a copy data frame to work with
age_purchase_df = pymoli_df.copy()
#create bins for age groups
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
binlabels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
agegroups2 = pd.cut(pymoli_df["Age"], bins, labels = binlabels)
#add age bracket column to age_purchase_df
age_purchase_df["Age Bracket"] = agegroups2
#groupby Age Bracket
age_purchase_df = age_purchase_df.groupby(["Age Bracket"])
#find total number of purchases by age group
age_purchase_count = age_purchase_df["Purchase ID"].count()
# find average purcahse price by age group
ave_purchase_price_by_age = age_purchase_df["Price"].mean().map("${:.2f}".format)
# find total purchase value by age group
total_purchase_value_by_age = age_purchase_df["Price"].sum().map("${:.2f}".format)
# find average total purchase amount per player by age group
age_group_totals = age_purchase_df["Price"].sum()
individuals_age = age_purchase_df["SN"].nunique()
ave_per_player_age = (age_group_totals/individuals_age).map("${:.2f}".format)
#create a new data frame to display the purchasing analysis by age
age_purchase_analysis_df = pd.DataFrame({"Purchase Count" : age_purchase_count, "Average Purchase Price" : ave_purchase_price_by_age,
                                         "Total Purchase Value" : total_purchase_value_by_age, "Avg Total Purchase per Person" : ave_per_player_age})
age_purchase_analysis_df.sort_index(inplace=True)
age_purchase_analysis_df



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Bracket,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [114]:
#top spenders
spender_df = pymoli_df.copy()

total_purchase = spender_df.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")

average_purchase = spender_df.groupby(["SN"]).mean()["Price"]
count_purchase = spender_df.groupby(["SN"]).count()["Price"].rename("Purchase Count")
spender_df = pd.DataFrame({ "Purchase Count" : count_purchase ,
                          "Average Purchase Price" : average_purchase, "Total Purchase Value" : total_purchase })


#format data
spender_df_final = spender_df.sort_values(by = ["Total Purchase Value"] , ascending = [False])
spender_df_final["Average Purchase Price"] = spender_df_final["Average Purchase Price"].map("${:.2f}".format)
spender_df_final["Total Purchase Value"] = spender_df_final["Total Purchase Value"].map("${:.2f}".format)

spender_df_final.head()



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [115]:
#Retrieve the Item ID, Item Name, and Item Price columns
#Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value

popular_df = pymoli_df.copy()
popular_df = popular_df.drop(["Purchase ID", "SN", "Age","Gender"],axis=1)

gpurchase_count = gender_grouped_df["Purchase ID"].count

popular_grouped = popular_df.groupby(["Item ID" , "Item Name"])

popular_count = popular_grouped["Item ID"].count()

popular_value = popular_grouped["Price"].sum()

pop_item_prices = popular_grouped["Price"].mean()

#create new dataframe
pop_analysis_df = pd.DataFrame({"Purchase Count":popular_count, "Price":pop_item_prices, "Total Purchase Value":popular_value})


In [108]:
#sort popular items by purchase count
pop_analysis_final = pop_analysis_df.sort_values(by = ["Purchase Count"] , ascending = False)
pop_analysis_final["Price"] = pop_analysis_final["Price"].map("${:.2f}".format)
pop_analysis_final["Total Purchase Value"] = pop_analysis_final["Total Purchase Value"].map("${:.2f}".format)
pop_analysis_final.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [109]:
#sort by most profitable using above dataframe
pop_analysis_df = pd.DataFrame({"Purchase Count":popular_count, "Price":pop_item_prices, "Total Purchase Value":popular_value})

In [116]:
#sort popular items by most profitable
pop_analysis_final = pop_analysis_df.sort_values(by = ["Total Purchase Value"] , ascending = False)
pop_analysis_final["Price"] = pop_analysis_final["Price"].map("${:.2f}".format)
pop_analysis_final["Total Purchase Value"] = pop_analysis_final["Total Purchase Value"].map("${:.2f}".format)

pop_analysis_final.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
